In [95]:
import pandas as pd
import pickle

In [96]:
test_data = pd.read_excel('data/Building_Strength_train_test.xlsx', sheet_name='test', engine='openpyxl')

test_data.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7,NaN
1,480.0,0.0,0.0,192.0,0.0,936.0,721.0,28,NaN
2,480.0,0.0,0.0,192.0,0.0,936.0,721.0,90,NaN
3,397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,NaN
4,281.0,0.0,0.0,186.0,0.0,1104.0,774.0,7,NaN


In [97]:
test_data.columns

Index(['cement', 'slag', 'flyash', 'water', 'superplasticizer',
       'coarseaggregate', 'fineaggregate', 'age', 'csMPa'],
      dtype='object')

In [98]:
# normalize the dataset

test_data_selected = test_data[test_data.columns[test_data.columns!='csMPa']].values


#td_mean = 298.6481917475728
#td_std = 352.3836093554314

td_mean = test_data_selected.mean()
td_std = test_data_selected.std()

print(f'Mean: {td_mean}')
print(f'Std: {td_std}')

Mean: 293.3337068965517
Std: 344.06802808463914


In [99]:
test_data_normalized = (test_data_selected - td_mean)/td_std
test_data_normalized

array([[ 0.01065572, -0.85254567, -0.85254567, ...,  2.1701124 ,
         1.28075339, -0.83220085],
       [ 0.54252728, -0.85254567, -0.85254567, ...,  1.86784659,
         1.24297016, -0.77116641],
       [ 0.54252728, -0.85254567, -0.85254567, ...,  1.86784659,
         1.24297016, -0.59096949],
       ...,
       [-0.42094497, -0.44739323, -0.53691041, ...,  1.74112746,
         1.41444788, -0.77116641],
       [-0.39013711, -0.30992042, -0.85254567, ...,  2.02362974,
         1.44031486, -0.77116641],
       [-0.09426539, -0.56045227, -0.62497439, ...,  1.66003885,
         1.36067944, -0.77116641]])

In [100]:
## load the model

model = pickle.load(open('model/concrete-strength-model-20210610.pickle.dat','rb'))
#model = pickle.load(open('model/concrete-strength-model-20210610-wo-normalization.pickle.dat','rb'))
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.001, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [101]:
predictions = model.predict(test_data_normalized)
#predictions = model.predict(test_data_selected)

In [102]:
## generate the output file

output = pd.DataFrame({'cement':test_data['cement'],
                       'slag':test_data['slag'],
                       'flyash': test_data['flyash'],
                       'water': test_data['water'],
                       'superplasticizer': test_data['superplasticizer'],
                       'coarseaggregate': test_data['coarseaggregate'],
                       'fineaggregate': test_data['fineaggregate'],
                       'age': test_data['age'],
                       'csMPa': predictions
                      })

output.to_csv('data/concrete_strength_prediction_output.csv', index=False)